In [ ]:
# 데이터 불러오기
import pandas as pd
import re 

pd.set_option('display.max_rows', 10000)

df = pd.read_csv(
    "./data/202512_202512_연령별인구현황_월간.csv",
    encoding="cp949"
)

df["지역명"] = df["행정구역"].str.split("(").str[0].str.strip()
df["행정코드"] = df["행정구역"].str.extract(r"\((\d+)\)").astype("Int64")

# 3. 시/군 단위만 추출
# - 시도: code % 100_000_000 == 0
# - 시군구: code % 100_000_000 != 0 AND code % 100_000 == 0
sigungu = df[
    (df["행정코드"] % 100_000_000 != 0) &
    (df["행정코드"] % 100_000 == 0)
].copy()

# 4. 숫자 컬럼 정리
num_cols = [
    "2025년12월_계_총인구수",
    "2025년12월_계_20~24세",
    "2025년12월_계_25~29세",
    "2025년12월_계_30~34세",
]

for c in num_cols:
    sigungu[c] = (
        sigungu[c]
        .astype(str)
        .str.replace(",", "", regex=False)
        .astype(int)
    )

# 5. 청년(20~34) 인구 계산
sigungu["청년(20~34)인구"] = (
    sigungu["2025년12월_계_20~24세"]
    + sigungu["2025년12월_계_25~29세"]
    + sigungu["2025년12월_계_30~34세"]
)

sigungu["청년비율(20~34)"] = (
    sigungu["청년(20~34)인구"]
    / sigungu["2025년12월_계_총인구수"]
)

# 6. 제외할 광역 단위 정의
exclude_regions = [
    "서울특별시", "경기도", 
    "인천광역시", "광주광역시", "울산광역시",
    "부산광역시", "대구광역시", "대전광역시"
]

sigungu["상위지역"] = sigungu["지역명"].str.split().str[0]

local_sigungu = sigungu[
    ~sigungu["상위지역"].isin(exclude_regions) & (sigungu["지역명"].str.split().str.len() == 2)
].copy()

# 7. 결과 테이블 구성
result = local_sigungu[
    ["지역명", "2025년12월_계_총인구수", "청년(20~34)인구", "청년비율(20~34)"]
].rename(columns={"2025년12월_계_총인구수": "총인구"})

result["청년비율(20~34)"] = (result["청년비율(20~34)"] * 100).round(2)

# 8. 결과 출력
print("=== [지방 시/군] 청년비율(20~34) 높은 지역 TOP 20 ===")
display(result.sort_values("청년비율(20~34)", ascending=False).head(50))

print("=== [지방 시/군] 청년비율(20~34) 낮은 지역 TOP 20 ===")
display(result.sort_values("청년비율(20~34)", ascending=True).head(50))

print("=== [지방 시/군] 청년인구(20~34) 많은 지역 TOP 20 ===")
display(result.sort_values("청년(20~34)인구", ascending=False).head(50))



In [ ]:
# 데이터 불러오기
import pandas as pd
import re 

pd.set_option('display.max_rows', 10000)

df = pd.read_csv(
    "../../data/202601_202601_주민등록인구및세대현황_월간.csv",
    encoding="cp949"
)

# 6. 제외할 광역 단위 정의
exclude_regions = [
    # "서울특별시", "경기도", 
    # "인천광역시", "광주광역시", "울산광역시",
    # "부산광역시", "대구광역시", "대전광역시"
]

df["지역명"] = df["행정구역"].str.split("(").str[0].str.strip()
df["행정코드"] = df["행정구역"].str.extract(r"\((\d+)\)").astype("Int64")

# 3. 시/군 단위만 추출
# - 시도: code % 100_000_000 == 0
# - 시군구: code % 100_000_000 != 0 AND code % 100_000 == 0
# sigungu = df[
#     (df["행정코드"] % 100_000_000 != 0) &
#     (df["행정코드"] % 100_000 == 0)
# ].copy()

sigungu = df.copy()

# 4. 숫자 컬럼 정리
num_cols = [
    "2026년01월_총인구수"
]

for c in num_cols:
    sigungu[c] = (
        sigungu[c]
        .astype(str)
        .str.replace(",", "", regex=False)
        .astype(int)
    )

# sigungu["상위지역"] = sigungu["지역명"].str.split().str[0]

# local_sigungu = sigungu[
#     ~sigungu["상위지역"].isin(exclude_regions) & (sigungu["지역명"].str.split().str.len() == 2)
# ].copy()

# # 7. 결과 테이블 구성
# result = local_sigungu[
#     ["지역명", "2026년01월_총인구수"]
# ].rename(columns={"2026년01월_총인구수": "총인구"})

# # 8. 결과 출력
# print("=== [지방 시/군] 인구로 정렬")

display(sigungu.sort_values("2026년01월_총인구수", ascending=False))

,행정구역,2026년01월_총인구수,2026년01월_세대수,2026년01월_세대당 인구,2026년01월_남자 인구수,2026년01월_여자 인구수,2026년01월_남여 비율,지역명,행정코드
0,서울특별시 (1100000000),"9,299,701","4,504,087",2.06,"4,480,849","4,818,852",0.93,서울특별시,1100000000
1,서울특별시 종로구 (1111000000),"136,798","72,274",1.89,"65,539","71,259",0.92,서울특별시 종로구,1111000000
2,서울특별시 중구 (1114000000),"117,718","63,815",1.84,"56,484","61,234",0.92,서울특별시 중구,1114000000
3,서울특별시 용산구 (1117000000),"201,746","102,893",1.96,"95,984","105,762",0.91,서울특별시 용산구,1117000000
4,서울특별시 성동구 (1120000000),"274,695","133,305",2.06,"132,585","142,110",0.93,서울특별시 성동구,1120000000
5,서울특별시 광진구 (1121500000),"330,933","171,161",1.93,"158,412","172,521",0.92,서울특별시 광진구,1121500000
6,서울특별시 동대문구 (1123000000),"348,620","177,750",1.96,"169,928","178,692",0.95,서울특별시 동대문구,1123000000
7,서울특별시 중랑구 (1126000000),"376,920","189,612",1.99,"184,035","192,885",0.95,서울특별시 중랑구,1126000000
8,서울특별시 성북구 (1129000000),"422,146","198,616",2.13,"201,710","220,436",0.92,서울특별시 성북구,1129000000
9,서울특별시 강북구 (1130500000),"278,750","142,306",1.96,"134,054","144,696",0.93,서울특별시 강북구,1130500000


,행정구역,2026년01월_총인구수,2026년01월_세대수,2026년01월_세대당 인구,2026년01월_남자 인구수,2026년01월_여자 인구수,2026년01월_남여 비율,지역명,행정코드
87,경기도 (4100000000),13736642,"6,128,139",2.24,"6,897,544","6,839,098",1.01,경기도,4100000000
0,서울특별시 (1100000000),9299701,"4,504,087",2.06,"4,480,849","4,818,852",0.93,서울특별시,1100000000
26,부산광역시 (2600000000),3239711,"1,573,947",2.06,"1,574,469","1,665,242",0.95,부산광역시,2600000000
262,경상남도 (4800000000),3205787,"1,549,092",2.07,"1,616,723","1,589,064",1.02,경상남도,4800000000
53,인천광역시 (2800000000),3053308,"1,401,983",2.18,"1,524,774","1,528,534",1.00,인천광역시,2800000000
237,경상북도 (4700000000),2503544,"1,295,172",1.93,"1,266,938","1,236,606",1.02,경상북도,4700000000
43,대구광역시 (2700000000),2352249,"1,113,217",2.11,"1,152,176","1,200,073",0.96,대구광역시,2700000000
179,충청남도 (4400000000),2136204,"1,062,477",2.01,"1,096,950","1,039,254",1.06,충청남도,4400000000
214,전라남도 (4600000000),1778171,"913,289",1.95,"898,237","879,934",1.02,전라남도,4600000000
197,전북특별자치도 (5200000000),1723497,"868,328",1.98,"858,643","864,854",0.99,전북특별자치도,5200000000
